In [3]:
import os
import cv2
import numpy as np
from skimage.filters import threshold_otsu
from skimage.feature import canny

def enhanced_robust_layout_extraction(image_path):
    # Step 0: Input
    I = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if I is None:
        print(f"⚠️  Lỗi đọc ảnh: {image_path}")
        return None
    I = cv2.normalize(I.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Step 1: Bilateral Filtering
    I_bilateral = cv2.bilateralFilter(I, d=5, sigmaColor=50, sigmaSpace=0.1)

    # Step 2: Gradient Computation (Scharr)
    Kx = np.array([[-3, 0, 3],
                   [-10, 0, 10],
                   [-3, 0, 3]], dtype=np.float32)
    Ky = np.array([[-3, -10, -3],
                   [0, 0, 0],
                   [3, 10, 3]], dtype=np.float32)
    Gx = cv2.filter2D(I_bilateral, -1, Kx)
    Gy = cv2.filter2D(I_bilateral, -1, Ky)
    E = np.sqrt(Gx**2 + Gy**2)
    E = cv2.normalize(E, None, 0, 1, cv2.NORM_MINMAX)

    # Step 3: Adaptive Threshold (Otsu)
    t_init = threshold_otsu(E)
    S = (E > t_init).astype(np.uint8)

    # Step 4: Multi-scale Non-Maximum Suppression (approx)
    edges_multi = (
        canny(I_bilateral, sigma=1.0) |
        canny(I_bilateral, sigma=2.0) |
        canny(I_bilateral, sigma=3.0)
    )
    S_refined = S & edges_multi.astype(np.uint8)

    # Step 5: Threshold Linking (Hysteresis)
    low_th = np.percentile(E, 90)
    high_th = np.percentile(E, 95)
    S_hyst = canny(I_bilateral, low_threshold=low_th, high_threshold=high_th)

    # Step 7: Output
    I_robust = (S_hyst * 255).astype(np.uint8)
    return I_robust


def process_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    supported_ext = ('.png', '.jpg', '.jpeg', '.tif', '.bmp')

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(supported_ext)]
    print(f"🔍 Tìm thấy {len(image_files)} ảnh trong thư mục: {input_folder}")

    for i, filename in enumerate(image_files, 1):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        print(f"[{i}/{len(image_files)}] 🔄 Đang xử lý: {filename}")
        result = enhanced_robust_layout_extraction(input_path)

        if result is not None:
            cv2.imwrite(output_path, result)
            print(f"✅ Lưu: {output_path}")
        else:
            print(f"❌ Bỏ qua: {filename} (không đọc được ảnh)")

    print("🎯 Hoàn tất xử lý tất cả ảnh!")


if __name__ == "__main__":
    # === Chỉnh đường dẫn tại đây ===
    input_dir = "/mnt/hdd1tb/SAR2Optical/DataSet/train/SAR"   # Thư mục ảnh đầu vào
    output_dir = "/mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/"  # Thư mục lưu kết quả

    process_folder(input_dir, output_dir)


🔍 Tìm thấy 441 ảnh trong thư mục: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR
[1/441] 🔄 Đang xử lý: F-48-58-C-b-1_2019-09-23_S1_p3.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-58-C-b-1_2019-09-23_S1_p3.png
[2/441] 🔄 Đang xử lý: F-48-58-C-b-1_2021-01-13_S1_p6.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-58-C-b-1_2021-01-13_S1_p6.png
[3/441] 🔄 Đang xử lý: F-48-58-C-b-1_2019-09-23_S1_p8.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-58-C-b-1_2019-09-23_S1_p8.png
[4/441] 🔄 Đang xử lý: F-48-93-C-a-3_2023-11-19_S1_p8.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-93-C-a-3_2023-11-19_S1_p8.png
[5/441] 🔄 Đang xử lý: F-48-68-D-d-1_2021-10-25_S1_p8.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-68-D-d-1_2021-10-25_S1_p8.png
[6/441] 🔄 Đang xử lý: F-48-92-D-a-3_2021-01-13_S1_p7.png
✅ Lưu: /mnt/hdd1tb/SAR2Optical/DataSet/train/SAR_BBDM/F-48-92-D-a-3_2021-01-13_S1_p7.png
[7/441] 🔄 Đang xử lý: F-48-57-D-c-4_2023-08-31_S

In [44]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
from skimage.morphology import disk, closing
from skimage.feature import canny

def show_step(image, title, cmap='gray'):
    plt.figure(figsize=(6,6))
    plt.imshow(image, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

def enhanced_robust_layout_extraction(image_path):
    # =========================
    # Step 0: Input
    # =========================
    I = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    I = cv2.normalize(I.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    # show_step(I, "Input Image (Normalized)")

    # =========================
    # Step 1: Preprocessing
    # =========================
    I_bilateral = cv2.bilateralFilter(I, d=5, sigmaColor=50, sigmaSpace=0.1)
    # show_step(I_bilateral, "Step 1: Bilateral Filtering")

    # =========================
    # Step 2: Gradient Computation (Scharr)
    # =========================
    Kx = np.array([[-3, 0, 3],
                   [-10, 0, 10],
                   [-3, 0, 3]], dtype=np.float32)
    Ky = np.array([[-3, -10, -3],
                   [0, 0, 0],
                   [3, 10, 3]], dtype=np.float32)

    Gx = cv2.filter2D(I_bilateral, -1, Kx)
    Gy = cv2.filter2D(I_bilateral, -1, Ky)
    E = np.sqrt(Gx**2 + Gy**2)
    E = cv2.normalize(E, None, 0, 1, cv2.NORM_MINMAX)
    # show_step(E, "Step 2: Gradient Magnitude")

    # =========================
    # Step 3: Adaptive Thresholding (Otsu)
    # =========================
    t_init = threshold_otsu(E)
    S = (E > t_init).astype(np.uint8)
    # show_step(S, f"Step 3: Adaptive Threshold (Otsu τ={t_init:.4f})")

    # =========================
    # Step 4: Multi-scale Non-Maximum Suppression
    # (approximation using multi-sigma Canny union)
    # =========================
    edges_multi = (
        canny(I_bilateral, sigma=1.0) |
        canny(I_bilateral, sigma=2.0) |
        canny(I_bilateral, sigma=3.0)
    )
    S_refined = S & edges_multi.astype(np.uint8)
    # show_step(S_refined, "Step 4: Multi-scale NMS")

    # =========================
    # Step 5: Threshold Linking (Hysteresis)
    # =========================
    # Adapt hysteresis threshold from E histogram
    low_th = np.percentile(E, 90)
    high_th = np.percentile(E, 95)
    S_hyst = canny(I_bilateral, low_threshold=low_th, high_threshold=high_th)
    # show_step(S_hyst, f"Step 5: Hysteresis Linking (low={low_th:.3f}, high={high_th:.3f})")

    # # =========================
    # # Step 6: Morphological Enhancement
    # # =========================
    # S_closed = closing(S_hyst, disk(3))
    # show_step(S_closed, "Step 6: Morphological Closing (Disk=3)")

    # =========================
    # Step 7: Output
    # =========================
    I_robust = (S_hyst * 255).astype(np.uint8)
    # show_step(I_robust, "Final Output: Enhanced Robust Layout Map")

    return I_robust
def process_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    supported_ext = ('.png', '.jpg', '.jpeg', '.tif', '.bmp')

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(supported_ext)]
    print(f"🔍 Tìm thấy {len(image_files)} ảnh trong thư mục: {input_folder}")

    for i, filename in enumerate(image_files, 1):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        print(f"[{i}/{len(image_files)}] 🔄 Đang xử lý: {filename}")
        result = enhanced_robust_layout_extraction(input_path)

        if result is not None:
            cv2.imwrite(output_path, result)
            print(f"✅ Lưu: {output_path}")
        else:
            print(f"❌ Bỏ qua: {filename} (không đọc được ảnh)")

    print("🎯 Hoàn tất xử lý tất cả ảnh!")

if __name__ == "__main__":
    # === Chỉnh đường dẫn tại đây ===
    input_dir = "/mnt/hdd1tb/Data_BBDM_GRM/test/EO"   # Thư mục ảnh đầu vào
    output_dir = "/mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/"  # Thư mục lưu kết quả

    process_folder(input_dir, output_dir)


🔍 Tìm thấy 108 ảnh trong thư mục: /mnt/hdd1tb/Data_BBDM_GRM/test/EO
[1/108] 🔄 Đang xử lý: F-48-68-C-d-2_2020-06-02_p3.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-68-C-d-2_2020-06-02_p3.png
[2/108] 🔄 Đang xử lý: F-48-68-C-d-2_2023-01-28_p3.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-68-C-d-2_2023-01-28_p3.png
[3/108] 🔄 Đang xử lý: F-48-68-C-c-2_2023-01-28_p5.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-68-C-c-2_2023-01-28_p5.png
[4/108] 🔄 Đang xử lý: F-48-68-C-d-3_2020-06-02_p5.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-68-C-d-3_2020-06-02_p5.png
[5/108] 🔄 Đang xử lý: F-48-80-A-c-1_2021-06-17_p7.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-80-A-c-1_2021-06-17_p7.png
[6/108] 🔄 Đang xử lý: F-48-79-B-b-4_2023-07-07_p2.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-79-B-b-4_2023-07-07_p2.png
[7/108] 🔄 Đang xử lý: F-48-67-D-b-1_2023-07-12_p8.png
✅ Lưu: /mnt/hdd1tb/Data_BBDM_GRM/test/SAR_BBDM/F-48-67-D-b-1_2023-07-12_p8.png
[